In [64]:
import numpy as np
import tensorflow as tf
from time import time
from matplotlib import pyplot as plt
import os

In [65]:
mydict = {'Ch': 0, 'Ma': 1, 'Na': 2, 'Sa': 3, 'Sh': 4}
import librosa
audio_file_path="C:\\Users\\ril\\Downloads\\sh_195.wav"
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)
mfccs = np.mean(librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40).T,axis=0)
x=[]
x.append(mfccs)
x=np.array(x)
x=np.reshape(x,(x.shape[0],10,4,1))

# PREDICT USING REGULAR KERAS TRAINED MODEL FILE (h5).

In [67]:

keras_model_size = os.path.getsize("D:\\shravanne-tasks\\Akshar classification\\Akshar_model.h5")/1048576  #Convert to MB
print("Keras Model size is: ", keras_model_size, "MB")
#Using regular keral model
model = tf.keras.models.load_model("D:\\shravanne-tasks\\Akshar classification\\Akshar_model.h5")

time_before=time()
keras_prediction = model.predict(x)
time_after=time()
keras_prediction= np.round_(keras_prediction)
a,b=np.where(keras_prediction==1)

total_keras_time = time_after - time_before
print("Total prediction time for keras model is: ", total_keras_time)
for gerne, classs in mydict.items(): 
    if classs == b[0]:
        print("prediction of keras model is: ",gerne)

Keras Model size is:  0.8233566284179688 MB
1/1 [==============================] - 1s 1s/step
Total prediction time for keras model is:  2.0541610717773438
prediction of keras model is:  Sh


# PREDICT USING tflite

In [68]:
tflite_size = os.path.getsize("D:\\shravanne-tasks\\Akshar classification\\Akshar_model.tflite")/1048576  #Convert to MB
print("tflite Model without opt. size is: ", tflite_size, "MB")
#Not optimized (file size = 540MB). Taking about 0.5 seconds for inference
tflite_model_path = "D:\\shravanne-tasks\\Akshar classification\\Akshar_model.tflite"
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()


tflite Model without opt. size is:  0.2656135559082031 MB


In [69]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [70]:
# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)


[ 1 10  4  1]


In [71]:
output_shape = output_details[0]['shape']
print(output_shape)

[1 5]


In [72]:
# Load image
input_data = x

interpreter.set_tensor(input_details[0]['index'], input_data)

time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_time = time_after - time_before
print("Total prediction time for tflite without opt model is: ", total_tflite_time)

output_data_tflite = interpreter.get_tensor(output_details[0]['index'])
print("The tflite w/o opt prediction for this image is: ", output_data_tflite)
output_data_tflite=np.round_(output_data_tflite)
a,b=np.where(output_data_tflite==1)
for gerne, classs in mydict.items(): 
    if classs == b[0]:
        print("\nTotal prediction time for keras model is: ",gerne)


Total prediction time for tflite without opt model is:  0.03789567947387695
The tflite w/o opt prediction for this image is:  [[8.4683555e-17 6.9707966e-17 4.5913492e-22 1.8663337e-11 1.0000000e+00]]

Total prediction time for keras model is:  Sh
